In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import preprocess as ppx

Using TensorFlow backend.


In [2]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [3]:
# epoch szám
nb_epoch=10

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót
x = base_model.output
x = GlobalAveragePooling2D()(x)
# ezután hozzáadunk egy előrecsatolt réteget ReLU aktivációs függvénnyel
x = Dense(1024, activation='relu')(x)
# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(14, activation='sigmoid')(x)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf


In [4]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='binary_crossentropy')

In [5]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names)
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

[[[189 180 165]
  [181 172 157]
  [189 180 165]
  ..., 
  [181 176 154]
  [164 159 137]
  [205 200 178]]

 [[197 188 173]
  [191 182 167]
  [189 180 165]
  ..., 
  [180 175 153]
  [184 179 157]
  [217 212 190]]

 [[197 188 173]
  [198 189 174]
  [190 181 166]
  ..., 
  [179 174 154]
  [198 193 173]
  [207 202 182]]

 ..., 
 [[ 96  17  22]
  [ 91  19  22]
  [ 74  13  10]
  ..., 
  [  3  18  11]
  [  3  18  13]
  [  5  20  15]]

 [[107  28  33]
  [ 96  24  27]
  [ 69   8   5]
  ..., 
  [  7  20  13]
  [ 10  23  16]
  [ 14  27  20]]

 [[114  35  40]
  [101  29  32]
  [ 75  14  11]
  ..., 
  [ 21  34  25]
  [ 21  34  27]
  [ 21  34  27]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ..., 
 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]



In [6]:
arr = np.array([np.asarray(ls[1]) for ls in train_images])
labels = np.array([ls[1] for ls in train_data])

encoder = preprocessing.LabelEncoder()
encoder.fit(labels)
encoded_l = encoder.transform(labels)

labels_onehot = to_categorical(encoded_l)

In [7]:
model.fit(np.asarray(train_images), labels_onehot,  batch_size=6, nb_epoch=10, callbacks=[history])

Epoch 1/10
14/14 [==============================] - 5s - loss: 0.6873     
Epoch 2/10
14/14 [==============================] - 4s - loss: 0.2239     
Epoch 3/10
14/14 [==============================] - 4s - loss: 0.0853     
Epoch 4/10
14/14 [==============================] - 4s - loss: 0.0593     
Epoch 5/10
14/14 [==============================] - 4s - loss: 0.0201     
Epoch 6/10
14/14 [==============================] - 4s - loss: 0.0219     
Epoch 7/10
14/14 [==============================] - 4s - loss: 0.0340     
Epoch 8/10
14/14 [==============================] - 4s - loss: 0.0185     
Epoch 9/10
14/14 [==============================] - 4s - loss: 0.0151     
Epoch 10/10
14/14 [==============================] - 4s - loss: 0.0101     


In [8]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

In [9]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(np.asarray(train_images), labels_onehot,  batch_size=6, nb_epoch=10, callbacks=[history])
print("Tanítás vége.")

Epoch 1/10
14/14 [==============================] - 6s - loss: 0.0142     
Epoch 2/10
14/14 [==============================] - 5s - loss: 0.0176     
Epoch 3/10
14/14 [==============================] - 5s - loss: 0.0127     
Epoch 4/10
14/14 [==============================] - 5s - loss: 0.0074     
Epoch 5/10
14/14 [==============================] - 5s - loss: 0.0117     
Epoch 6/10
14/14 [==============================] - 5s - loss: 0.0199     
Epoch 7/10
14/14 [==============================] - 5s - loss: 0.0134     
Epoch 8/10
14/14 [==============================] - 5s - loss: 0.0396     
Epoch 9/10
14/14 [==============================] - 5s - loss: 0.0117     
Epoch 10/10
14/14 [==============================] - 5s - loss: 0.0207     
Tanítás vége.


In [10]:
for pred in model.predict(np.asarray(train_images)):
    for p in pred:
        print('%.2f' % p)
    print()

0.00
0.44
0.00
0.90
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.01
0.00

0.00
0.57
0.00
0.00
0.00
0.00
0.01
1.00
0.00
0.03
0.00
0.00
0.06
0.06

0.00
0.01
0.00
0.00
1.00
0.02
0.01
0.00
0.00
0.00
0.00
0.03
0.02
0.37

0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
1.00

0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
1.00
0.00
0.00
0.00
0.00

0.00
0.00
0.04
0.00
0.00
0.00
0.00
0.00
1.00
0.00
0.00
0.00
0.00
0.00

0.00
0.20
1.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00

0.00
1.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00

0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.97
0.00

0.00
0.04
0.00
0.00
0.00
1.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00

0.00
0.00
0.00
0.00
0.00
0.00
0.99
0.00
0.00
0.00
0.00
0.00
0.00
0.00

1.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00

0.00
0.01
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.90
0.00
0.03

0.00
0.16
0.01
0.00
0.00
0.01
0.00
0.00
0.00
0.00
0.75
0.04
0.00
0.00

